### RAG 기본 구조 이해하기

1. 사전작업(Pre-processing): 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 

- 1단계 문서로드(Document Load): 문서 내용을 불러옴
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장

2. RAG 수행(RunTime) - 5~8 단계

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됨. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정 가능
- 7단계 LLM: 모델을 정의 (GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성

## 환경설정


In [1]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv("./.env", override=True)

True

In [4]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
os.environ['LANGCHAIN_PROJECT'] = 'RAG'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[API KEY]
sk-ABp1f2C1BWf8SkXnrEil9hP3OvsxTmnytRq4mM6Z1aT3BlbkFJiR43shY2AF75_rIKJAP4HqGY35yCJ82Ha7r-XYW1sA
[LANGCHAIN_PROJECT]
RAG


In [5]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG")


LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


## 네이버 뉴스 기반 QA(Question-Answering) 챗봇

네이버 뉴스기사의 내용에 대해 질문할 수 있는 **뉴스기사 QA 앱** 을 구축

In [7]:
### PDF 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("초보 투자자를 위한 증권과 투자 따라잡기.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 73

[페이지내용]
018 019초보 투자자를 위한 증권과 투자 따라잡기 증권시장의 이해
 한국증권거래소
통합 이전 한국증권거래소는 주식, 채권, KOSPI 200 주가지수선물 및 
옵션 등이 활발하게 거래되는 증권시장으로 거래규모 및 시가총액이 세계 
10위권에 이르렀습니다.
1956년 비영리법인인 대한증권거래소가 설립되었다가 1963년 정부 및 
증권회사가 공동출자한 공영제 조직인 한국증권거래소로 다시 개장하게 
되었습니다. 1988년 3월부터는 증권회사를 회원으로 하는 회원제 조직의 
사단법인으로 개편하게 되었습니다.
유가증권의 원활한 유통과 공정한 가격형성을 위해 1971년의 포스트
매매제도를 도입한 이후, 1987년 포스트매매와 전산매매를 병행하다가 
1997년 9월부터 전 종목을 전산시스템으로 매매하고 있습니다. 유가증권
의 매매에 따른 결제업무를 원활히 하고 체계적인 유가증권의 보관 및 관
리를 위해 1974년 현재 한국예탁결제원의 전신인 증권대체결제주식회사
를 설립하였고, 증권시장의 효율

[metadata]
{'producer': 'iLovePDF', 'creator': 'Adobe InDesign CC 14.0 (Macintosh)', 'creationdate': '2022-02-03T13:49:31+09:00', 'trapped': '/False', 'moddate': '2024-06-18T14:13:06+00:00', 'source': '초보 투자자를 위한 증권과 투자 따라잡기.pdf', 'total_pages': 73, 'page': 10, 'page_label': '11'}



In [ ]:
### csv 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 로드
loader = CSVLoader(file_path="data/titanic.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

In [ ]:
### 폴더 내의 모든 파일 로드하여 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")


### 폴더 내의 모든 pdf 로드하여 QA(Question-Answering) 챗봇으로 변경하는 코드
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.pdf")
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[2500:3000])

In [ ]:
### Python 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import PythonLoader

loader = DirectoryLoader(".", glob="**/*.py", loader_cls=PythonLoader)
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

In [ ]:
### txt 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/appendix-keywords.txt")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

In [8]:
!pip install bs4


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import bs4  # 크롤링
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [11]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/296/0000082139",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/296/0000082139'}, page_content='\n"애벌레처럼 뭘 입는거야?"...몸 압박해 꿀잠 잔다? 뭔가 봤더니\n\n\n틱톡에서 유행 중인 \'어른 포대기\'...머리부터 발끝까지 감싸고, 태아 자세로 누운 채로 애벌레처럼 뒹굴면 잠 잘온다 주장\n\n\n\n천으로 몸을 감싼 후 잠자리에 드는 새로운 수면법이 트렌드로 떠오르고 있다. 맨 오른쪽=일본의 전통 치료법인 \'오토나마키\' [사진=영국 일간 데일리메일 보도 갈무리]잠 잘자는 묘책, 애벌레가 되어라?  천으로 몸을 감싼 후 잠자리에 드는 새로운 수면법이 트렌드로 떠오르고 있다. 아기를 천에 감싸는 것처럼 자신의 몸을 감싸는 방식으로 마치 애벌레를 연상케 한다.  틱톡에서는 신축성 있는 천에 몸을 구겨 넣고 잠자리에 드는 장면을 어렵지 않게 찾을 수 있다. 이렇게 몸을 감싸고 자면 불안 완화, 자세 개선, 깊은 수면 등 건강상의 이점을 제공한다는 것이 이들의 주장이다. 머리부터 발끝까지 감싸고, 태아 자세로 누운 채 부드럽게 흔들리거나 굴러다니면 잠을 잘 자게 돕는다는 것.  이 트렌드를 옹호하는 사람들은 몸을 천으로 감싸는 자체가 피부 깊숙한 층의 촉각 수용체를 자극해 긴장을 풀어준다고 입을 모은다. 실제로 특정 신경 세포가 활성화되면 평온한 느낌을 촉진하는 것으로 알려져 있긴하다. 한 여성은 이 자세를 통해 오랜 불면증을 해결했다고 주장했다.  실상 이 아이디어는 일본의 전통 치료법인 \'오토나마키\'에서 유래된 것으로 여겨진다는 것이 전문가들의 설명이다. 오토나마키는 직역하면 \'어른 포대기\'로, 사람들이 머리부터 발끝까지 큰 천으로 감싸는 방법이다. 원래 산후 재활 치료의 일환으로 개발됐다. 출산 후 산모의 신체 유연성 개선과 근육 이완, 관절 통증 완화 목적에서 일반적인 신체적 긴장 완화 및 유연성 개선을 위한 방법으로 그 개념이 확장됐다.  \'어른 포대기\'는

`RecursiveCharacterTextSplitter`는 문서를 지정된 크기의 청크로 나눔


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

`FAISS` 혹은 `Chroma`와 같은 vectorstore는 이러한 청크를 바탕으로 문서의 벡터 표현을 생성


In [13]:
# 벡터스토어를 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

`vectorstore.as_retriever()`를 통해 생성된 검색기는 프롬프트와 `ChatOpenAI` 모델을 사용하여 새로운 내용을 생성

`StrOutputParser`는 생성된 결과를 문자열로 파싱


In [14]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [15]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

스트리밍 출력을 위하여 `stream_response` 를 사용

In [16]:
from langchain_teddynote.messages import stream_response

In [17]:
answer = rag_chain.stream("새로운 수면법을 알려줘..")
stream_response(answer)

새로운 수면법으로는 천으로 몸을 감싸고 잠자리에 드는 방법이 있습니다. 이 방법은 일본의 전통 치료법인 '오토나마키'에서 유래된 것으로, 머리부터 발끝까지 큰 천으로 몸을 감싸는 방식입니다. 이렇게 몸을 감싸고 태아 자세로 누운 채로 부드럽게 흔들리거나 굴러다니면 불안 완화, 자세 개선, 깊은 수면 등의 건강상의 이점을 제공한다고 주장합니다. 이 방법은 피부 깊숙한 층의 촉각 수용체를 자극해 긴장을 풀어주는 효과가 있다고 알려져 있습니다. 다만, 이 방법이 불면증을 해결하는 데 대한 과학적 증거는 아직 부족하다고 합니다.

In [18]:
answer = rag_chain.stream("뉴스기사의 새로운 수면법을 찾아서 이를 영어로 번역해줘.")
stream_response(answer)

새로운 수면법은 천으로 몸을 감싸고 잠자리에 드는 방식으로, 일본의 전통 치료법인 '오토나마키'에서 유래된 것으로 알려져 있습니다. 이 방법은 몸을 감싸는 것이 피부 깊숙한 층의 촉각 수용체를 자극하여 긴장을 풀어주고, 불안 완화, 자세 개선, 깊은 수면 등의 건강상의 이점을 제공한다고 주장합니다. 

이 수면법을 영어로 번역하면 다음과 같습니다:

"A new sleeping method involves wrapping the body in fabric before going to bed, reminiscent of the Japanese traditional therapy 'otonamaki'. This method is said to provide health benefits such as anxiety relief, posture improvement, and deep sleep by stimulating the tactile receptors deep within the skin."

In [19]:
answer = rag_chain.stream("새로운 수면 법을 bullet points 형식으로 작성해 주세요.")
stream_response(answer)

- 새로운 수면법은 천으로 몸을 감싸고 잠자리에 드는 방식이다.
- 이 방법은 아기를 천에 감싸는 것처럼 자신의 몸을 감싸는 형태로, 애벌레를 연상시킨다.
- 틱톡에서는 신축성 있는 천에 몸을 구겨 넣고 자는 장면이 많이 공유되고 있다.
- 몸을 감싸고 자면 불안 완화, 자세 개선, 깊은 수면 등의 건강상의 이점이 있다고 주장된다.
- 머리부터 발끝까지 감싸고 태아 자세로 누워 부드럽게 흔들리거나 굴러다니면 잠을 잘 자게 돕는다.
- 이 방법은 일본의 전통 치료법인 '오토나마키'에서 유래된 것으로 여겨진다.
- '오토나마키'는 산후 재활 치료의 일환으로 개발되었으며, 신체 유연성 개선과 근육 이완, 관절 통증 완화에 도움을 준다.
- 이 수면법은 '깊은 압박 터치(DTP, Deep Touch Pressure)' 원칙에 따른다.
- DTP는 신경과학에서 감각 과부하나 불안증을 완화하는 데 효과적이라는 연구 결과가 있다.
- 그러나 DTP가 불면증을 해결한다는 과학적 증거는 아직 없다.

In [21]:
answer = rag_chain.stream("대한민국 대통령을 알려줘")
stream_response(answer)

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.

### [실습] RAG를 활용하여 네이버 뉴스기사 QA 챗봇 만들기
- 아래 전체 코드에서 네이버 뉴스 기사를 넣어 원하는 기사 QA 챗봇 실습 해보기
- 기사 내용에 담겨 있는 질문을 넣고 챗봇이 올바르게 기사의 내용을 답변하는지 살펴보기
- 기사 외의 내용을 질문하여 할루시네이션 현상이 나타나는지 확인하기
- LangSmith에서 관련된 청크를 잘 찾아오는지 확인해보기

##### 뉴스 기사 챗봇을 생성하는 전체 코드

In [ ]:
# 1. 뉴스기사 내용을 로드
loader = WebBaseLoader(
    web_paths=("네이버 뉴스 기사 링크",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

# 2. 불러온 뉴스 기사를 청크로 나누고, 인덱싱
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

# 3. 벡터스토어를 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 4. 뉴스에 포함되어 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


# 5. 체인을 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer = rag_chain.stream("기사에 있는 내용을 포함한 질문 넣기")
stream_response(answer)

In [ ]:
answer = rag_chain.stream("기사에 있는 내용을 포함한 질문 넣기")
stream_response(answer)

In [ ]:
answer = rag_chain.stream("기사에 없는 내용을 포함한 질문 넣기")
stream_response(answer)